# (Attempt) to extract data from Bloomberg using an API

## Set Up 

### Import Modules

In [2]:
import pandas as pd
import numpy as np
import blpapi
import sys
import os
from optparse import OptionParser
import xbbg
from xbbg import blp

__file__ = "NB03_Extract_Bloomberg_Data.ipynb"
try:
    current_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, '..', '..', 'src'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    print(f'Inserting Parent Path: {parent_dir}')

# Print the updated sys.path
print(f'Priority System Path: {sys.path[0]}')

# Try to import the custom module
try:
    import econ_utils
    from econ_utils import inv_theory as inv
    print('Successfully imported inv_theory as inv from econ_utils')
except ImportError as e:
    print(f'Error importing econ_utils: {e}')
    sys.exit(1)

Inserting Parent Path: c:\Users\HALMALVA\OneDrive\Desktop\EconProjects\src
Priority System Path: c:\Users\HALMALVA\OneDrive\Desktop\EconProjects\src
Current Directory: c:\Users\HALMALVA\OneDrive\Desktop\EconProjects\Notebooks\Investment_Theory
Successfully imported inv_theory as inv from econ_utils


### Set up Data Directory

In [3]:
DATA_DIR = os.path.join(sys.path[0],"..", "data")
print(f'Data_Directory: {DATA_DIR}')
#df = pd.read_excel(os.path.join(DATA_DIR, "Top_Stocks.xlsx"))

Data_Directory: c:\Users\HALMALVA\OneDrive\Desktop\EconProjects\src\..\data


### Choose Starting Data and Collect a BDH Request

In [4]:
tickers = ["NVDA US Equity", "AAPL US Equity"]
fields = ["Last_Price"]
start_date = "2023-09-01"
end_date = "2023-09-20"

hist_tick_data = blp.bdh(tickers=tickers, flds = fields, start_date=start_date, end_date=end_date)
print("Collected BDS (Bulk Data) Data")
filename = f'tick_data_{start_date}_{end_date}.csv'
hist_tick_data.to_csv(os.path.join(DATA_DIR, filename))

Collected BDS (Bulk Data) Data


## But we want to do a BEQS (Bloomberg Equity Screening)

The API's BEQS Function is not fantastic. We will just get the tickers from an excel bloomberg query and re-use them whenever we need.

In [28]:
tickers = list(pd.read_excel('../../data/tickers.xlsx').ID.values)
#tickers

### Select Our Set of Fields

In [38]:
flds = ['Security_Name', 'px_last', 'current_trr_ytd', 'cur_mkt_cap', 'eps_growth', 'pr_eps_growth_plus_yld', 'geo_grow_diluted_eps_cont_ops', 'msci_st_forward_eps_growth_rate', 'diluted_eps_cont_ops_5yr_avg_gr', '5y_geo_growth_diluted_eps', 'earn_yld','pe_ratio', 'five_yr_avg_price_earnings', 'long_term_price_earnings_ratio', 'best_pe_ratio', 'est_per_nxt_yr', 'current_trr_ytd', 'current_ann_trr_3yr', 'current_ann_trr_5yr', 'month_end_trr_10yr','minimum_total_return', 'hist_trr_prev_1yr', 'current_trr_mtd']

In [39]:
bloomberg_data = blp.bdp(tickers = tickers, flds=flds)
bloomberg_data[:20]

,security_name,px_last,current_trr_ytd,cur_mkt_cap,current_trr_ytd_1,current_ann_trr_3yr,current_ann_trr_5yr,month_end_trr_10yr,minimum_total_return,hist_trr_prev_1yr,...,eps_growth,geo_grow_diluted_eps_cont_ops,diluted_eps_cont_ops_5yr_avg_gr,5y_geo_growth_diluted_eps,earn_yld,pe_ratio,five_yr_avg_price_earnings,long_term_price_earnings_ratio,best_pe_ratio,pr_eps_growth_plus_yld
000333 CH Equity,Midea Group Co Ltd,64.0,22.18981,446727409791.999939,22.18981,1.145894,6.88653,21.26763,NaN,10.85621,...,11.248249,9.349578,9.054256,9.621721,7.78053,12.852595,16.435892,18.269357,NaN,NaN
000660 KS Equity,SK Hynix Inc,236000.0,67.06522,171808558140000.03125,67.06522,26.32282,29.94964,18.93422,NaN,90.69447,...,NaN,NaN,NaN,NaN,-2.836933,NaN,NaN,29.344968,7.878897,42.571356
000858 CH Equity,Wuliangye Yibin Co Ltd,128.95,-8.096358,500533352244.749939,-8.096358,-20.54755,1.532926,24.22911,NaN,-20.57978,...,13.190809,17.636014,5.300772,17.50692,6.335673,15.783643,33.542675,28.014891,NaN,13.64292
005930 KS Equity,Samsung Electronics Co Ltd,87100.0,11.45114,519968060105000.0625,11.45114,4.956467,17.32351,14.68361,NaN,44.89824,...,-73.550949,-19.810062,78.530824,-19.895532,3.329506,30.034482,19.193506,17.522041,13.515566,63.123398
1088 HK Equity,China Shenhua Energy Co Ltd,37.35,49.26952,925645000883.896973,49.26952,50.08257,31.52436,15.39087,NaN,32.60036,...,-11.362398,7.717313,10.502693,7.950673,8.938004,11.18818,6.399881,14.706056,NaN,NaN
1211 HK Equity,BYD Co Ltd,235.0,11.20932,729271130537.50769,11.20932,2.926014,38.61143,18.01099,NaN,11.85827,...,80.735552,63.919901,350.658468,61.822665,4.792254,20.867006,87.911764,81.603832,NaN,10.55968
1288 HK Equity,Agricultural Bank of China Ltd,3.39,20.98979,1634762207710.838135,20.98979,17.6543,9.18704,6.759664,NaN,22.66104,...,3.905365,5.582058,2.855729,5.45946,23.94637,4.175998,3.873082,4.802827,NaN,8.881938
1299 HK Equity,AIA Group Ltd,51.75,-22.45382,579621171701.25,-22.45382,-16.32979,-7.605971,4.755203,NaN,-19.95864,...,17.048711,18.607601,54.040546,8.674462,4.926925,20.296634,25.921124,16.070176,10.912825,20.868068
1305 JP Equity,iFree ETF TOPIX,3089.0,23.58472,11152798000000.0,23.58472,17.22507,15.41314,10.7474,-4.6867,27.85505,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1306 JP Equity,NEXT FUNDS TOPIX Exchange Trad,3045.0,23.05516,24523676000000.0,23.05516,17.14256,15.34945,10.72992,-4.85254,28.11915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save this data out as excel

In [40]:
bloomberg_data.to_excel("../../data/bloomberg_data.xlsx")

## Once Raw Data Has been Extracted:

In [7]:
print("hello")

hello
